# 提取符合规则的词对

In [1]:
import pickle
import collections
candidateFea = set()
candidateFeaOpinion = set()
candidate_FO_pair = list()
fea_to_opinion = collections.defaultdict(list)
opinion_to_fea = collections.defaultdict(list)

## 规则

In [2]:
#名词动词主谓关系，并列关系
rule1 = [('n','v','SBV'),('v','a','COO')]
#名词形容词主谓关系
rule2 = [['n','a','SBV'],['a','n','SBV']]
#动补关系
rule3 = [['v','a','CMP'],['a','v','CMP']]
#并列关系
rule4 = [['a','a','COO'],['i','a','COO'],['a','i','COO']]

## 根据规则提取词对

In [3]:
def processRules123(line):
    words_list = []
    pos_relate_list = []
    return_pair = []
    #规则1
    for pair in line:
        if pair:
            pair = pair[1:-1].split('_')
            words_list.append((pair[0],pair[2]))
            pos_relate_list.append((pair[1],pair[3],pair[4]))
            #print(len(pair))
            #print(type(pair))
            #print(pair)
    #print(words_list)
    #print(pos_relate_list)
    
    for word_pair_pos_pair in zip(words_list, pos_relate_list):
        
        word_pair = word_pair_pos_pair[0]
        pos_pair = word_pair_pos_pair[1]
        hit_noun = []
        hit_verb = []
        if pos_pair == rule1[0]:
            #print('yes')
            
            hit_noun = word_pair[0]
            hit_verb = word_pair[1]
            return_pair.append((hit_noun, hit_verb))
            #print(return_pair)
            #print(hit_noun,hit_verb)
    #print(return_pair)
        elif pos_pair == rule1[1] and hit_noun and hit_verb:
            return_pair.append((hit_noun,word_pair[1]))
    #规则2        
    for pair in line:
        if pair:
            pair = pair[1:-1].split('_')
            rules = [pair[1],pair[3],pair[4]]
            for id, r in enumerate(rule2):
                if rules == r:
                    if id == 0:
                        return_pair.append((pair[0],pair[2]))
                    else:
                        return_pair.append((pair[2],pair[0]))
    #规则3                    
    for pair in line:
        if pair:
            pair = pair[1:-1].split('_')
            rules = [pair[1],pair[3],pair[4]]
            for id, r in enumerate(rule3):
                if rules == r:
                    if id == 0:
                        return_pair.append((pair[0],pair[2]))
                    else:
                        return_pair.append((pair[2],pair[0]))
        #print(return_pair)
    #print(set(return_pair))
    return list(set(return_pair))
    #print(list(set(return_pair)))

In [4]:
def processRules4(line):
    return_list = []
    for pair in line:
        if pair:
            pair = pair[1:-1].split('_')
            rule = [pair[1],pair[3],pair[4]]
            if rule in rule4:
                return_list.append(pair[0])
                return_list.append(pair[2])
    return list(set(return_list))

In [5]:
def extractWord(filename):
    with open(filename,mode='r',encoding='UTF-8') as f:
        for line in f:
            #print(line)
            #print(type(line))
            #print(line)
            currentPair = list()
            line = line.strip(',').replace('\n','')
            line = line.split(',')
            line[0] = line[0].replace('\ufeff','')
            #print(line)
            #print(line[0])
            rules_list = processRules123(line)
            #print(rules_list)
            #print(n)
            if rules_list:
                for pair in rules_list:
                    candidateFea.add(pair[0])
                    candidateFeaOpinion.add(pair[1])
                    currentPair.append((pair[0],pair[1]))
                    fea_to_opinion[pair[0]].append(pair[1])
                    opinion_to_fea[pair[1]].append(pair[0])
            #成语等一系列形容词        
            opinion_list = processRules4(line)
            if opinion_list:
                for word in opinion_list:
                    candidateFeaOpinion.add(word)
                    
            candidate_FO_pair.append(currentPair)

In [6]:
def savePairToFile(data, file):
    data = list(data)
    with open(file,'w') as f:
        for eachline in data:
            for each in eachline:
                f.write(each[0])
                f.write(':')
                f.write(each[1])
                f.write(',')
            f.write('\n')
                

In [7]:
def saveToFile(data, file):
    data = list(data)
    with open(file, 'w') as f:
        for ele in data:
            f.write(str(ele))
            f.write('\n')

In [8]:
def saveDict(data, file):
    with open(file, 'w') as f:
        for key, value in data.items():
            f.write(key + ':[')
            f.write(','.join(value))
            f.write(']' + '\n')

In [9]:
if __name__ =='__main__':
    filename = 'F:/data2/rules.txt'
    #filename = 'F:/data2/testRules.txt'
    outfilePath = 'F:/data2/'
    extractWord(filename)
    savePairToFile(candidate_FO_pair, outfilePath+'candidate_FO_pair.txt')
    saveToFile(candidateFea, outfilePath+'candidateFea.txt')
    saveToFile(candidateFeaOpinion, outfilePath+'candidateFeaOpinion.txt')
    saveDict(fea_to_opinion, outfilePath+'fea_to_opinion.txt')
    saveDict(opinion_to_fea, outfilePath+'opinion_to_fea.txt')
    with open(outfilePath + 'fea_to_opinion.pkl', mode='wb+') as f:
        pickle.dump(fea_to_opinion, f)
    with open(outfilePath + 'opinion_to_fea.pkl', mode='wb+') as f:
        pickle.dump(opinion_to_fea, f)
    